In [1]:
import requests as req
import http.client
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
from fuzzywuzzy import process, fuzz


/home/kike/anaconda3/envs/Entorno_Virtual_BT_Data_Science/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Datos que queremos necesitamos.

Tabla de Equipos de La Liga. ( Id. Nombre Equipo) .

Tabla de Jugadores. 

(Id Jugador, Id del equipo, Posición, Partidos Jugados 21/22, Goles, Asistencias, Puntos Futmondo, Puntos Fantasy, Precio de mercado Futmondo, Puntos totales temporada 20/21 Futmondo, Puntos totales temporada 20/21 Fantasy).

Fuentes: 

Tabla Equipos desde API

Jugadores scraping desde espn.

Puntuación de futmondo. web scraping página futmondo.



In [2]:


connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': '2643e9e1f8624a75a8eb21f12ad5c0bf' }
connection.request('GET', '/v2/competitions/PD/teams?season=2021', None, headers )
response = json.loads(connection.getresponse().read().decode())['teams']#[0]['name']



In [3]:
team_lst = [response[t]['name'] for t in range(len(response))]
team_lst
team_id = [i for i in range(1,21)]


In [4]:
df_teams = pd.DataFrame(team_lst, columns = ['Team'])
df_teams.insert(0, 'Team_id', team_id)

In [5]:
df_teams.head()

,Team_id,Team
0,1,Athletic Club
1,2,Club Atlético de Madrid
2,3,CA Osasuna
3,4,RCD Espanyol de Barcelona
4,5,FC Barcelona


In [6]:
df_teams.to_csv('teams.csv', index=False)

In [ ]:
teams_urls = ['']
ath = [teams_lst[0], team_id[0], 'https://www.comuniate.com/plantilla/1/athletic',]

In [ ]:
# función tiene como parámetros (team, team_id, url, numero_judadores, url2)

In [6]:
#url='https://www.espn.com/soccer/team/squad/_/id/96/esp.alaves'
url= 'https://www.comuniate.com/plantilla/89/alaves' # para casi todos los datos
url2 = 'https://www.comuniazo.com/comunio-apuestas/equipos/alaves' # para los ptos ultimas jornadas

soup = bs(req.get(url).text , 'html.parser')
soup2 = bs(req.get(url2).text , 'html.parser')

jugadores = 25 # necesario porque no todas las plantillas tienes los mismos jugadores

#cuerpo de la función
soup = bs(req.get(url).text , 'html.parser')

team_name = soup.find_all('strong')[20].text.split()[2:]

team_id = team_id

equipo = team_name * jugadores

name = soup.find_all('strong')
name = [name[i].text for i in range(22, (22 + jugadores * 2), 2)]

pos_gk = soup.find_all('span', class_='label-posicion label-success')
gks = [pos_gk[i].text.strip() for i in range(len(pos_gk))]

pos_df = soup.find_all('span', class_='label-posicion label-info')
dfs = [pos_df[i].text.strip() for i in range(len(pos_df))]

pos_md = soup.find_all('span', class_='label-posicion label-warning')
mds = [pos_md[i].text.strip() for i in range(len(pos_md))]

pos_fw = soup.find_all('span', class_='label-posicion label-danger')
fws = [pos_fw[i].text.strip() for i in range(len(pos_fw))]

all_pos = gks + dfs + mds + fws

pts = soup.find_all('span', class_='texto_pequenio2 label-posicion label-primary')

pt = [pts[i].text for i in range(len(pts))]

values = soup.find_all('small')
value = [values[i].text.split('€')[0] for i in range(0,len(values),3)]

pt_avg = [values[i].text.split()[-1] for i in range(1,len(values),3)]

on_start_per = [values[i].text.split('%')[0] for i in range(2,len(values),3)]

puntos = soup2.find_all('div', class_='streak')
pts_last_five_jouyneys = [puntos[i].text.split() for i in range(len(puntos))] 

team = {#'Team_id': team_id, #entra como parametro en la función
        'Team': equipo, # cambiar por el parametro de entrada de entrada team
        'Player': name,
        'Position': all_pos,
        'Total Points': pt,
        'Points Average': pt_avg,
        'Value': value,
        'On start %': on_start_per,
       'Last 5 matchs points': pts_last_five_jouyneys }

#print(equipo)
#print(len(name))
#print(len(all_pos))
#print(len(pt))
#print(team)

#df = pd.DataFrame(team)
#df
df = pd.DataFrame(team)
df.to_csv(f'{team_name}.csv', index = False)
df


,Team,Player,Position,Total Points,Points Average,Value,On start %,Last 5 matchs points
0,Alavés,Pacheco,PT,51,4.6,2.970.000,85,"[8, 5, -, 8, 4]"
1,Alavés,Sivera,PT,8,2.7,350.000,15,"[-, 2, 4, -, -]"
2,Alavés,Laguardia,DF,65,5.4,3.360.000,92,"[9, 9, 6, 3, 7]"
3,Alavés,Rubén Duarte,DF,43,3.6,2.720.000,92,"[3, 7, 0, 7, 3]"
4,Alavés,Miazga,DF,35,4.4,1.600.000,46,"[4, -, -, -, 1]"
5,Alavés,Ximo Navarro,DF,29,3.2,1.000.000,69,"[3, 5, 1, -, -]"
6,Alavés,Martín Aguirregabiria,DF,27,3.4,1.070.000,46,"[-, 4, 1, 6, 0]"
7,Alavés,Lejeune,DF,26,3.7,2.890.000,54,"[-, 6, 6, 6, 0]"
8,Alavés,Alberto Rodríguez,DF,5,2.5,370.000,8,"[-, -, -, -, -]"
9,Alavés,Javi López Carballo,DF,4,2,420.000,8,"[-, -, -, -, -]"


In [7]:
pts_last_5_matchs = soup.find_all('div', class_='col-md-12')
pts_last_5_matchs[3].text.split('\n')[83:88]
#pacheco = pts_last_5_matchs[2].text.split('\n')[25:30]
#pacheco[::-1]
#sivera = pts_last_5_matchs[2].text.split('\n')[53:58]
#sivera[::-1]
#laguardia = pts_last_5_matchs[3].text.split('\n')[25:30]
#duarte = pts_last_5_matchs[3].text.split('\n')[54:59]

['4', '-', '-', '-', '1']

In [8]:
url2 = 'https://www.comuniazo.com/comunio-apuestas/equipos/alaves'
soup2 = bs(req.get(url2).text , 'html.parser')
puntos = soup2.find_all('div', class_='streak')

#for i in range(len(puntos)):
#    print(puntos[i].text.split())

pts_last_five_jouyneys = [puntos[i].text.split() for i in range(len(puntos))] 
pts

[<span class="texto_pequenio2 label-posicion label-primary">51</span>,
 <span class="texto_pequenio2 label-posicion label-primary">8</span>,
 <span class="texto_pequenio2 label-posicion label-primary">65</span>,
 <span class="texto_pequenio2 label-posicion label-primary">43</span>,
 <span class="texto_pequenio2 label-posicion label-primary">35</span>,
 <span class="texto_pequenio2 label-posicion label-primary">29</span>,
 <span class="texto_pequenio2 label-posicion label-primary">27</span>,
 <span class="texto_pequenio2 label-posicion label-primary">26</span>,
 <span class="texto_pequenio2 label-posicion label-primary">5</span>,
 <span class="texto_pequenio2 label-posicion label-primary">4</span>,
 <span class="texto_pequenio2 label-posicion label-primary">3</span>,
 <span class="texto_pequenio2 label-posicion label-primary">0</span>,
 <span class="texto_pequenio2 label-posicion label-primary">43</span>,
 <span class="texto_pequenio2 label-posicion label-primary">43</span>,
 <span clas

In [9]:
espn_url = 'https://espndeportes.espn.com/futbol/equipo/estadisticas/_/id/96/alaves'

soup = bs(req.get(espn_url).text , 'html.parser')

team_name = soup.find('span', class_='db fw-bold').text.strip()

goals = soup.find('tbody', class_='Table__TBODY')

players = goals.find_all('tr')

player_stats = [players[i].text for i in range(len(players)-1)]

all_stats = []

for p in player_stats:
    #print(p)
    name = ''
    stats = []
    p_stats = []
    for i in range(len(p)):
       
        if p[i].isnumeric() and  i == 0:
            #print(p[i])
            #print('Es numero y el primer caracter')
            continue
        elif p[i].isalpha() or p[i] == ' ':
            #print(p[i])
            #print('es letra')
            name += p[i]
        elif p[i].isnumeric():
            stats.append(p[i])
            #print(p[i])
            #print('es numero')
    p_stats.append(name)
    p_stats.append(stats)
    matchs_played = ''
    goals = ''
    
    if len(stats) == 2:
        matchs_played += stats[0]
        goals += stats[1]
    if len(stats) == 3:
        matchs_played += stats[0]
        matchs_played += stats[1]
        goals += stats[2]
    if len(stats) == 4:
        matchs_played += stats[0]
        matchs_played += stats[1]
        goals += stats[2]
        goals += stats[3]
    #print(matchs_played)
    #print(goals)
    
    all_stats_dict = {
        'Team': team_name,
        'Player': name,
        'Matchs': matchs_played,
        'Goals': goals
    }
    all_stats.append(all_stats_dict)
    #print(name, stats)
    
print(all_stats)

[{'Team': 'Alavés', 'Player': 'Joselu', 'Matchs': '13', 'Goals': '6'}, {'Team': 'Alavés', 'Player': 'Víctor Laguardia', 'Matchs': '12', 'Goals': '2'}, {'Team': 'Alavés', 'Player': 'Luis Rioja', 'Matchs': '13', 'Goals': '1'}, {'Team': 'Alavés', 'Player': 'Mamadou Loum', 'Matchs': '13', 'Goals': '1'}, {'Team': 'Alavés', 'Player': 'Facundo Pellistri', 'Matchs': '12', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Antonio Moya', 'Matchs': '12', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Manu García', 'Matchs': '12', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Rubén Duarte', 'Matchs': '12', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Tomás Pina', 'Matchs': '12', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Fernando Pacheco', 'Matchs': '11', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Édgar Méndez', 'Matchs': '9', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Ximo Navarro', 'Matchs': '9', 'Goals': '0'}, {'Team': 'Alavés', 'Player': 'Martín Aguirregabiria', 'Matchs': '8', 'Goals': '0'}, {'T

In [10]:
df_stats = pd.DataFrame(all_stats)


,Team,Player,Matchs,Goals
22,Alavés,Alberto Rodríguez,2,0
5,Alavés,Antonio Moya,12,0
19,Alavés,Antonio Sivera,3,0
20,Alavés,Barbero,6,0
10,Alavés,Edgar Méndez,9,0
4,Alavés,Facundo Pellistri,12,0
9,Alavés,Fernando Pacheco,11,0
18,Alavés,Florian Lejeune,7,0
23,Alavés,Iván Martín,1,0
21,Alavés,Javier López,2,0


In [26]:
espn_url = 'https://espndeportes.espn.com/futbol/equipo/estadisticas/_/id/96/alaves'

soup = bs(req.get(espn_url).text , 'html.parser')

team_name = soup.find('span', class_='db fw-bold').text.strip()

assistances = soup.find('tbody', class_='Table__TBODY')

players = assistances.find_all('tr', class_='Table__TR Table__TR--sm Table__even')

player_stats2 = [players[i].text for i in range(len(players)-1)]

player_stats2

all_stats2 = []

for p in player_stats2:
    #print(p)
    name = ''
    stats = []
    p_stats = []
    for i in range(len(p)):
       
        if p[i].isnumeric() and  i == 0:
            #print(p[i])
            #print('Es numero y el primer caracter')
            continue
        elif p[i].isalpha() or p[i] == ' ':
            #print(p[i])
            #print('es letra')
            name += p[i]
        elif p[i].isnumeric():
            stats.append(p[i])
            #print(p[i])
            #print('es numero')
    p_stats.append(name)
    p_stats.append(stats)
    matchs_played = ''
    assistances = ''
    
    if len(stats) == 2:
        matchs_played += stats[0]
        assistances += stats[1]
    if len(stats) == 3:
        matchs_played += stats[0]
        matchs_played += stats[1]
        assistances += stats[2]
    if len(stats) == 4:
        matchs_played += stats[0]
        matchs_played += stats[1]
        assistances += stats[2]
        assistances += stats[3]
    #print(matchs_played)
    #print(goals)
    
    all_stats2_dict = {
        'Team': team_name,
        'Player': name,
        'Matchs': matchs_played,
        'Assintances': assistances
    }
    all_stats2.append(all_stats2_dict)
    #print(name, stats)
    
print(all_stats2)


[{'Team': 'Alavés', 'Player': 'Joselu', 'Matchs': '13', 'Assintances': '6'}, {'Team': 'Alavés', 'Player': 'Víctor Laguardia', 'Matchs': '12', 'Assintances': '2'}, {'Team': 'Alavés', 'Player': 'Luis Rioja', 'Matchs': '13', 'Assintances': '1'}, {'Team': 'Alavés', 'Player': 'Mamadou Loum', 'Matchs': '13', 'Assintances': '1'}, {'Team': 'Alavés', 'Player': 'Facundo Pellistri', 'Matchs': '12', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Antonio Moya', 'Matchs': '12', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Manu García', 'Matchs': '12', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Rubén Duarte', 'Matchs': '12', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Tomás Pina', 'Matchs': '12', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Fernando Pacheco', 'Matchs': '11', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Édgar Méndez', 'Matchs': '9', 'Assintances': '0'}, {'Team': 'Alavés', 'Player': 'Ximo Navarro', 'Matchs': '9', 'Assintances': '0'}, {'Team': 'Alavé

In [27]:
df_stats2 = pd.DataFrame(all_stats2)
df_stats2

,Team,Player,Matchs,Assintances
0,Alavés,Joselu,13,6
1,Alavés,Víctor Laguardia,12,2
2,Alavés,Luis Rioja,13,1
3,Alavés,Mamadou Loum,13,1
4,Alavés,Facundo Pellistri,12,0
5,Alavés,Antonio Moya,12,0
6,Alavés,Manu García,12,0
7,Alavés,Rubén Duarte,12,0
8,Alavés,Tomás Pina,12,0
9,Alavés,Fernando Pacheco,11,0


In [14]:
df[['team_matched', 'fuzz_score']]=df.Player.apply(lambda x:process.extractOne(x,
                                                                     df_stats.Player.tolist(),
                                                                     scorer=fuzz.partial_ratio)).apply(pd.Series)
df=pd.merge(df, df_stats, left_on='team_matched', right_on='Player')
df_1=df[df.fuzz_score>85]
#df_1 = df1.drop(columns = ['team_matched', 'fuzz_score', 'Team_y', 'Player_y'])
df_1


,Team_x,Player_x,Position,Total Points,Points Average,Value,On start %,Last 5 matchs points,team_matched,fuzz_score,Team_y,Player_y,Matchs,Goals
0,Alavés,Pacheco,PT,51,4.6,2.970.000,85,"[8, 5, -, 8, 4]",Fernando Pacheco,100,Alavés,Fernando Pacheco,11,0
1,Alavés,Sivera,PT,8,2.7,350.000,15,"[-, 2, 4, -, -]",Antonio Sivera,100,Alavés,Antonio Sivera,3,0
2,Alavés,Laguardia,DF,65,5.4,3.360.000,92,"[9, 9, 6, 3, 7]",Víctor Laguardia,100,Alavés,Víctor Laguardia,12,2
3,Alavés,Rubén Duarte,DF,43,3.6,2.720.000,92,"[3, 7, 0, 7, 3]",Rubén Duarte,100,Alavés,Rubén Duarte,12,0
4,Alavés,Miazga,DF,35,4.4,1.600.000,46,"[4, -, -, -, 1]",Matt Miazga,100,Alavés,Matt Miazga,8,0
5,Alavés,Ximo Navarro,DF,29,3.2,1.000.000,69,"[3, 5, 1, -, -]",Ximo Navarro,100,Alavés,Ximo Navarro,9,0
6,Alavés,Martín Aguirregabiria,DF,27,3.4,1.070.000,46,"[-, 4, 1, 6, 0]",Martín Aguirregabiria,100,Alavés,Martín Aguirregabiria,8,0
7,Alavés,Lejeune,DF,26,3.7,2.890.000,54,"[-, 6, 6, 6, 0]",Florian Lejeune,100,Alavés,Florian Lejeune,7,0
8,Alavés,Alberto Rodríguez,DF,5,2.5,370.000,8,"[-, -, -, -, -]",Alberto Rodríguez,100,Alavés,Alberto Rodríguez,2,0
11,Alavés,Manu García Alonso,MD,42,3.5,1.420.000,38,"[1, 4, 4, 4, 4]",Manu García,100,Alavés,Manu García,12,0


In [16]:
df_1 = df_1.drop(columns = ['team_matched', 'fuzz_score', 'Team_y', 'Player_y'])
df_1

,Team_x,Player_x,Position,Total Points,Points Average,Value,On start %,Last 5 matchs points,Matchs,Goals
0,Alavés,Pacheco,PT,51,4.6,2.970.000,85,"[8, 5, -, 8, 4]",11,0
1,Alavés,Sivera,PT,8,2.7,350.000,15,"[-, 2, 4, -, -]",3,0
2,Alavés,Laguardia,DF,65,5.4,3.360.000,92,"[9, 9, 6, 3, 7]",12,2
3,Alavés,Rubén Duarte,DF,43,3.6,2.720.000,92,"[3, 7, 0, 7, 3]",12,0
4,Alavés,Miazga,DF,35,4.4,1.600.000,46,"[4, -, -, -, 1]",8,0
5,Alavés,Ximo Navarro,DF,29,3.2,1.000.000,69,"[3, 5, 1, -, -]",9,0
6,Alavés,Martín Aguirregabiria,DF,27,3.4,1.070.000,46,"[-, 4, 1, 6, 0]",8,0
7,Alavés,Lejeune,DF,26,3.7,2.890.000,54,"[-, 6, 6, 6, 0]",7,0
8,Alavés,Alberto Rodríguez,DF,5,2.5,370.000,8,"[-, -, -, -, -]",2,0
11,Alavés,Manu García Alonso,MD,42,3.5,1.420.000,38,"[1, 4, 4, 4, 4]",12,0


In [30]:
df_1[['team_matched', 'fuzz_score']]=df_1.Player_x.apply(lambda x:process.extractOne(x,
                                                                     df_stats2.Player.tolist(),
                                                                     scorer=fuzz.partial_ratio)).apply(pd.Series)
df_1=pd.merge(df_1, df_stats2, left_on='team_matched', right_on='Player')
df_2=df_1[df.fuzz_score>85]
df_2

/tmp/ipykernel_9380/2203609815.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_2=df_1[df.fuzz_score>85]


,Team_x,Player_x,Position,Total Points,Points Average,Value,On start %,Last 5 matchs points,Matchs_x,Goals,team_matched,fuzz_score,Team,Player,Matchs_y,Assintances
0,Alavés,Pacheco,PT,51,4.6,2.970.000,85,"[8, 5, -, 8, 4]",11,0,Fernando Pacheco,100,Alavés,Fernando Pacheco,11,0
1,Alavés,Sivera,PT,8,2.7,350.000,15,"[-, 2, 4, -, -]",3,0,Antonio Sivera,100,Alavés,Antonio Sivera,3,0
2,Alavés,Laguardia,DF,65,5.4,3.360.000,92,"[9, 9, 6, 3, 7]",12,2,Víctor Laguardia,100,Alavés,Víctor Laguardia,12,2
3,Alavés,Rubén Duarte,DF,43,3.6,2.720.000,92,"[3, 7, 0, 7, 3]",12,0,Rubén Duarte,100,Alavés,Rubén Duarte,12,0
4,Alavés,Miazga,DF,35,4.4,1.600.000,46,"[4, -, -, -, 1]",8,0,Matt Miazga,100,Alavés,Matt Miazga,8,0
5,Alavés,Ximo Navarro,DF,29,3.2,1.000.000,69,"[3, 5, 1, -, -]",9,0,Ximo Navarro,100,Alavés,Ximo Navarro,9,0
6,Alavés,Martín Aguirregabiria,DF,27,3.4,1.070.000,46,"[-, 4, 1, 6, 0]",8,0,Martín Aguirregabiria,100,Alavés,Martín Aguirregabiria,8,0
7,Alavés,Lejeune,DF,26,3.7,2.890.000,54,"[-, 6, 6, 6, 0]",7,0,Florian Lejeune,100,Alavés,Florian Lejeune,7,0
8,Alavés,Alberto Rodríguez,DF,5,2.5,370.000,8,"[-, -, -, -, -]",2,0,Alberto Rodríguez,100,Alavés,Alberto Rodríguez,2,0
11,Alavés,Toni Moya,MD,43,3.6,1.550.000,62,"[2, 1, 11, 3, 2]",12,0,Antonio Moya,89,Alavés,Antonio Moya,12,0


In [33]:
df_2 = df_2.drop(columns = ['team_matched', 'fuzz_score', 'Team', 'Player', 'Matchs_y'])


KeyError: "['team_matched' 'fuzz_score' 'Team' 'Player' 'Matchs_y'] not found in axis"

In [35]:
df_2 = df_2.rename(columns = {'Team_x': 'Team', 'Player_x': 'Player', 'Matchs_x': 'Matchs'})

In [36]:
df_2

,Team,Player,Position,Total Points,Points Average,Value,On start %,Last 5 matchs points,Matchs,Goals,Assintances
0,Alavés,Pacheco,PT,51,4.6,2.970.000,85,"[8, 5, -, 8, 4]",11,0,0
1,Alavés,Sivera,PT,8,2.7,350.000,15,"[-, 2, 4, -, -]",3,0,0
2,Alavés,Laguardia,DF,65,5.4,3.360.000,92,"[9, 9, 6, 3, 7]",12,2,2
3,Alavés,Rubén Duarte,DF,43,3.6,2.720.000,92,"[3, 7, 0, 7, 3]",12,0,0
4,Alavés,Miazga,DF,35,4.4,1.600.000,46,"[4, -, -, -, 1]",8,0,0
5,Alavés,Ximo Navarro,DF,29,3.2,1.000.000,69,"[3, 5, 1, -, -]",9,0,0
6,Alavés,Martín Aguirregabiria,DF,27,3.4,1.070.000,46,"[-, 4, 1, 6, 0]",8,0,0
7,Alavés,Lejeune,DF,26,3.7,2.890.000,54,"[-, 6, 6, 6, 0]",7,0,0
8,Alavés,Alberto Rodríguez,DF,5,2.5,370.000,8,"[-, -, -, -, -]",2,0,0
11,Alavés,Toni Moya,MD,43,3.6,1.550.000,62,"[2, 1, 11, 3, 2]",12,0,0
